In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics.cluster import entropy
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import RandomizedSearchCV, HalvingGridSearchCV
from sklearn.svm import SVC

In [ ]:
!curl -O https://raw.githubusercontent.com/rromer07/kickstarter-status-prediction/main/kickstarter_data_full.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 94.8M  100 94.8M    0     0  23.7M      0  0:00:03  0:00:03 --:--:-- 23.7M


In [ ]:
!curl -O https://raw.githubusercontent.com/rromer07/kickstarter-status-prediction/main/kickstarter_data_sampled_final.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9812k  100 9812k    0     0  16.1M      0 --:--:-- --:--:-- --:--:-- 16.1M


In [ ]:
df = pd.read_csv('kickstarter_data_sampled_final.csv',header=0)

In [ ]:
df.isna().sum()

slug                             5
created_at_month                 5
launch_to_state_change           5
LaunchedTuesday                  5
goal                             5
create_to_launch                 5
id_new                           2
created_at_hr                    5
deadline                         5
state_changed_at_month           5
deadline_weekday                 5
created_at                       5
category                       238
backers_count                    5
create_to_launch_days            5
deadline_hr                      5
state_changed_at_hr              5
country                          5
deadline_month                   5
staff_pick                       5
blurb                            2
launch_to_state_change_days      5
created_at_weekday               5
launched_at_weekday              5
launched_at_day                  5
name                             2
disable_communication            5
state_changed_at                 5
USorGB              

In [ ]:
display(df)

,slug,created_at_month,launch_to_state_change,LaunchedTuesday,goal,create_to_launch,id_new,created_at_hr,deadline,state_changed_at_month,...,launched_at,urls,DeadlineWeekend,state_changed_at_weekday,state_changed_at_day,launch_to_deadline_days,SuccessfulBool,created_at_day,launched_at_hr,additional description
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,the-nan-no-ad-network,2,30 days 00:00:01.000000000,1,50000.0,4 days 09:35:18.000000000,5646,2.0,3/12/15 12:22,3,...,2/10/15 12:22,"{""web"":{""project"":""N"",""rewards"":""https://www.k...",0,Thursday,12,30.0,0.0,6,12.0,NaN
2,mister-halo-a-c-misting-system,7,45 days 05:06:24.000000000,0,33000.0,64 days 18:53:18.000000000,17465,17.0,10/25/14 17:48,10,...,9/10/14 12:41,"{""web"":{""project"":""https://www.kickstarter.com...",1,Saturday,25,45.0,0.0,7,12.0,We are raising funds to pay a willing and able...
3,howcomo-action-sports-learn-share-and-get-spon...,7,2 days 18:47:14.000000000,1,200000.0,33 days 05:48:11.000000000,7037,12.0,9/13/16 17:54,8,...,8/16/16 17:54,"{""web"":{""project"":""https://www.kickstarter.com...",0,Friday,19,28.0,0.0,14,17.0,HowComo The first online application bringing ...
4,your-online-action-buddy-to-help-you-get-thing...,3,30 days 00:00:02.000000000,0,38000.0,17 days 18:31:56.000000000,6872,1.0,5/14/16 20:15,5,...,4/14/16 20:15,"{""web"":{""project"":""https://www.kickstarter.com...",1,Saturday,14,30.0,0.0,28,20.0,The Concept: Your Action Buddy. Your online p...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,onlineapp-make-own-web-applications-within-min...,4,30 days 00:00:02.000000000,0,3240,10 days 22:17:35.000000000,6563,13.0,5/31/16 11:32,5,...,5/1/16 11:32,"{""web"":{""project"":""https://www.kickstarter.com...",0,Tuesday,31,30.0,0.0,20,11.0,\nWhat is OnlineApp?OnlineApp is a web applica...
2497,starbot-animatronic-robot-puppets,7,30 days 00:00:50.000000000,1,5000,97 days 07:56:48.000000000,12003,9.0,11/21/13 17:14,11,...,10/22/13 17:14,"{""web"":{""project"":""https://www.kickstarter.com...",0,Thursday,21,30.0,1.0,17,17.0,The Star★Bot animatronic platform kit teaches ...
2498,hybmenu-create-your-digital-pizza-meal-engage,2,60 days 00:00:03.000000000,0,85000,3 days 07:40:10.000000000,10817,10.0,4/14/15 18:12,4,...,2/13/15 18:12,"{""web"":{""project"":""https://www.kickstarter.com...",0,Tuesday,14,60.0,0.0,10,18.0,"We called it ""HybMenu"" and it is a software fo..."
2499,utenants,7,30 days 00:00:01.000000000,1,100000,3 days 18:05:43.000000000,11171,15.0,8/4/16 9:50,8,...,7/5/16 9:50,"{""web"":{""project"":""https://www.kickstarter.com...",0,Thursday,4,30.0,0.0,1,9.0,- To help renters work together to resolve lan...


In [ ]:
# drop the ids and urls columns
df = df.drop(columns=['id_new', 'urls'])

# drop the 'garbage' rows
df = df.drop([0, 2340, 2341, 2342, 2466, 2467])

# fix the dtypes
dtypes = {'goal': float, 'disable_communication': bool, 'staff_pick': bool, 'backers_count': int, 'deadline_month': int, 'deadline_day': int, 'deadline_hr': int, 'state_changed_at_month': int, 'state_changed_at_day': int, 'state_changed_at_hr': int, 'created_at_month': int, 'created_at_day': int, 'created_at_hr': int, 'launched_at_month': int, 'launched_at_day': int, 'launched_at_hr': int, 'create_to_launch_days': int, 'launch_to_deadline_days': int, 'launch_to_state_change_days': int, 'SuccessfulBool': int, 'USorGB': int, 'TOPCOUNTRY': int, 'LaunchedTuesday': int, 'DeadlineWeekend': int}
df = df.astype(dtypes)


df = df.drop(columns=['state_changed_at','disable_communication','deadline_weekday','deadline_month','launched_at_month','DeadlineWeekend','additional description','slug','blurb','name','USorGB','TOPCOUNTRY','LaunchedTuesday','create_to_launch','deadline','created_at','launched_at','launched_at_hr'])


In [ ]:
display(df)

,created_at_month,launch_to_state_change,goal,created_at_hr,state_changed_at_month,category,backers_count,create_to_launch_days,deadline_hr,state_changed_at_hr,...,created_at_weekday,launched_at_weekday,launched_at_day,launch_to_deadline,deadline_day,state_changed_at_weekday,state_changed_at_day,launch_to_deadline_days,SuccessfulBool,created_at_day
1,2,30 days 00:00:01.000000000,50000.0,2,3,Web,1,4,12,12,...,Friday,Tuesday,10,30 days 00:00:00.000000000,12,Thursday,12,30,0,6
2,7,45 days 05:06:24.000000000,33000.0,17,10,Gadgets,11,64,17,17,...,Monday,Wednesday,10,45 days 05:06:24.000000000,25,Saturday,25,45,0,7
3,7,2 days 18:47:14.000000000,200000.0,12,8,Web,6,33,17,12,...,Thursday,Tuesday,16,28 days 00:00:00.000000000,13,Friday,19,28,0,14
4,3,30 days 00:00:02.000000000,38000.0,1,5,Web,2,17,20,20,...,Monday,Thursday,14,30 days 00:00:00.000000000,14,Saturday,14,30,0,28
5,11,30 days 00:00:00.000000000,5000.0,3,7,Hardware,7,232,14,14,...,Monday,Tuesday,23,30 days 00:00:00.000000000,23,Thursday,23,30,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,4,30 days 00:00:02.000000000,3240.0,13,5,Web,1,10,11,11,...,Wednesday,Sunday,1,30 days 00:00:00.000000000,31,Tuesday,31,30,0,20
2497,7,30 days 00:00:50.000000000,5000.0,9,11,Robots,98,97,17,17,...,Wednesday,Tuesday,22,30 days 00:00:00.000000000,21,Thursday,21,30,1,17
2498,2,60 days 00:00:03.000000000,85000.0,10,4,Software,0,3,18,18,...,Tuesday,Friday,13,60 days 00:00:00.000000000,14,Tuesday,14,60,0,10
2499,7,30 days 00:00:01.000000000,100000.0,15,8,Software,2,3,9,9,...,Friday,Tuesday,5,30 days 00:00:00.000000000,4,Thursday,4,30,0,1


In [ ]:
#@title Preprocessing categorical variables
# deadline_weekday, category, country, staff_pick, created_at_weekday, launched_at_weekday, disable_communication, state_changed_at_weekday

In [ ]:
# Create a label encoder object
encoder = LabelEncoder()

# Fit the encoder to the labels and transform the labels
df['category'] = encoder.fit_transform(df['category'])
df['country'] = encoder.fit_transform(df['country'])
df['created_at_weekday'] = encoder.fit_transform(df['created_at_weekday'])
df['launched_at_weekday'] = encoder.fit_transform(df['launched_at_weekday'])
df['launch_to_state_change'] = encoder.fit_transform(df['launch_to_state_change'])
df['launch_to_deadline'] = encoder.fit_transform(df['launch_to_deadline'])
df['state_changed_at_weekday'] = encoder.fit_transform(df['state_changed_at_weekday'])


print(df['category'].value_counts())
print(df['country'].value_counts())
print(df['created_at_weekday'].value_counts())
print(df['launched_at_weekday'].value_counts())


7     400
21    333
6     277
16    275
22    233
12    190
1     159
10    134
20    111
4      70
14     69
17     67
8      39
5      37
3      34
18     30
9      23
11      6
15      4
2       1
0       1
19      1
13      1
Name: category, dtype: int64
20    1715
9      317
3      136
1       81
5       49
15      37
8       32
12      22
7       20
4       12
11      11
6       11
17       9
16       8
14       8
18       7
10       7
0        5
2        3
19       3
13       2
Name: country, dtype: int64
5    437
6    414
4    398
1    389
0    326
3    283
2    248
Name: created_at_weekday, dtype: int64
5    565
1    502
6    433
4    382
0    354
3    133
2    126
Name: launched_at_weekday, dtype: int64


In [ ]:
# target variable 
y = df['SuccessfulBool']

In [ ]:
# train/test split for numerical data
X= df.drop(['SuccessfulBool'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1996, 22), (499, 22), (1996,), (499,))

In [ ]:
display(df)

,created_at_month,launch_to_state_change,goal,created_at_hr,state_changed_at_month,category,backers_count,create_to_launch_days,deadline_hr,state_changed_at_hr,...,created_at_weekday,launched_at_weekday,launched_at_day,launch_to_deadline,deadline_day,state_changed_at_weekday,state_changed_at_day,launch_to_deadline_days,SuccessfulBool,created_at_day
1,2,444,50000.0,2,3,21,1,4,12,12,...,0,5,10,277,12,4,12,30,0,6
2,7,842,33000.0,17,10,6,11,64,17,17,...,1,6,10,576,25,2,25,45,0,7
3,7,162,200000.0,12,8,21,6,33,17,12,...,4,5,16,193,13,0,19,28,0,14
4,3,445,38000.0,1,5,21,2,17,20,20,...,1,4,14,277,14,2,14,30,0,28
5,11,443,5000.0,3,7,7,7,232,14,14,...,1,5,23,277,23,4,23,30,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,4,445,3240.0,13,5,21,1,10,11,11,...,6,3,1,277,31,5,31,30,0,20
2497,7,470,5000.0,9,11,14,98,97,17,17,...,6,5,22,277,21,4,21,30,1,17
2498,2,956,85000.0,10,4,16,0,3,18,18,...,5,0,13,680,14,5,14,60,0,10
2499,7,444,100000.0,15,8,16,2,3,9,9,...,0,5,5,277,4,4,4,30,0,1


##XGBoost Hyperparamter

In [ ]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

xgb_model = xgb.XGBClassifier()

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2]
}

grid_search_xg = GridSearchCV(xgb_model, param_grid, cv=5, n_jobs=-1)
grid_search_xg.fit(X_train, y_train)

print("Best parameters:", grid_search_xg.best_params_)
print("Best score:", grid_search_xg.best_score_)

Best parameters: {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
Best score: 0.9288659147869673


In [ ]:
y_pred = grid_search_xg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test accuracy:", accuracy)
print(classification_report(y_test, y_pred))

Test accuracy: 0.9178356713426854
              precision    recall  f1-score   support

           0       0.93      0.90      0.91       240
           1       0.91      0.94      0.92       259

    accuracy                           0.92       499
   macro avg       0.92      0.92      0.92       499
weighted avg       0.92      0.92      0.92       499



In [ ]:
# #@title Hyperparam tuning SVM
param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'],'gamma': [0.1, 1, 10]}

# Create an SVC object
svc = SVC()

# Create a GridSearchCV object
random_search_svc = HalvingGridSearchCV(svc, param_distributions=param_grid,refit=True,verbose=100,n_jobs=-1,cv=3,random_state = 42)


# Fit the GridSearchCV object to the data
random_search_svc.fit(X_train ,y_train)


# Print the best parameters and the corresponding score
print("Best parameters: ", random_search_svc.best_params_)
print("Best score: ", random_search_svc.best_score_)

y_pred = random_search_svc.predict(X_test)

# calculate accuracy score of best model on test set
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy score: ", accuracy)
print(classification_report(y_test, y_pred))

In [ ]:
#@title Hyperparam tuning Bernouli
from sklearn.naive_bayes import BernoulliNB

nb = BernoulliNB(alpha=1.0, binarize=0.0)

from sklearn.model_selection import GridSearchCV

param_grid = {'alpha': [0.1, 1.0, 10.0]}
grid_search_nb = GridSearchCV(nb, param_grid, cv=5)
grid_search_nb.fit(X_train, y_train)

print("Best alpha:", grid_search_nb.best_params_['alpha'])
print("Best validation score:", grid_search_nb.best_score_)

from sklearn.metrics import accuracy_score

nb = BernoulliNB(alpha=0.1, binarize=0.0)
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test accuracy:", accuracy)

Best alpha: 0.1
Best validation score: 0.5972117794486216
Test accuracy: 0.6112224448897795


##Random Forest

In [ ]:
param_grid_rf = {
    'n_estimators': [10, 50, 100, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf = RandomForestClassifier()
grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, cv=5,n_jobs = -1)

# Fit the model to the data
grid_search_rf.fit(X_train, y_train)


# Print the best parameters and score
print("Best Parameters: ", grid_search_rf.best_params_)
print("Best Score: ", grid_search_rf.best_score_)
best_rf = grid_search_rf.best_estimator_
y_pred = best_rf.predict(X_test)
print(classification_report(y_test, y_pred))

Best Parameters:  {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}
Best Score:  0.9188383458646616
              precision    recall  f1-score   support

           0       0.92      0.89      0.91       240
           1       0.90      0.93      0.92       259

    accuracy                           0.91       499
   macro avg       0.91      0.91      0.91       499
weighted avg       0.91      0.91      0.91       499

